In [ ]:
# !pip install pysqlite3
# !pip install tqdm

In [1]:
import numpy as np
import pandas as pd
from numpy import random
import sqlite3

import re

import random

import warnings
warnings.filterwarnings('ignore')


cnxn = sqlite3.connect("reccar.db")
cursor = cnxn.cursor() 

seed = 1998

In [15]:
import os
import sys
import urllib.request
from tqdm import tqdm
import time

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.chrome.options import Options

client_id = "789Xk04GARJpb4omVvUq" # 개발자센터에서 발급받은 Client ID 값
client_secret = "oynUXBN1cW" # 개발자센터에서 발급받은 Client Secret 값

In [3]:
area_list = ['Dobong', 'Dongdaemun', 'Dongjag', 'eunpyeong', 'geumcheon',
'Guro', 'GwanUck', 'Jongno', 'Junggu', 'Jungnang', 'Kongbuk',
'Kongdong', 'Kongjin', 'Kongnam', 'Kongseo', 'Mapo', 'Nowon', 'Seocho',
'Seodaemun', 'Seongbuk', 'Seongdong', 'songpa', 'Yangcheon',
'Yeongdeungpo', 'Yongsan']

len(area_list)

25

In [4]:
data = pd.DataFrame()
rest = pd.DataFrame()

cnt = 0
for area in area_list:
    tem = pd.read_csv(f'./user/{area}.csv')
    tem['userid'] = tem['link'].apply(lambda x: str(re.findall(r'my/[a-z0-9]+/', x))[5:-3])
    tem = tem[['userid', 'rest']]
    tem = tem.drop_duplicates()
    data = pd.concat([data, tem])

    try: tem = pd.read_csv(f'./rest/rest_concat_{area}.csv')
    except: continue
    tem = tem.drop_duplicates()
    #print(area)
    cnt += 1
    rest = pd.concat([rest, tem])   

data = data.drop_duplicates() # 30만개정도 빠짐
data = data[~data['rest'].isnull()] # 7개정도 null 값이 있더라
data.reset_index(drop=True, inplace = True)
rest.reset_index(drop=True, inplace = True)
print(cnt)

25


In [29]:
rest['image'] = rest['image'].fillna('tem')

rest['x'][rest['x'] == 'start'] = '-1'
rest['y'][rest['y'] == 'items'] = '-1'
rest['x'] = rest['x'].astype('int')
rest['y'] = rest['y'].astype('int')

In [41]:
def make_tag(x):
    tem = x.split(',')
    return tem

def cnt_tag(x):
    return len(x)

rest['tag'] = rest['tag'].apply(lambda x : make_tag(x))
rest['tag_len'] = rest['tag'].apply(lambda x : cnt_tag(x))
rest['tag_len'].max() # 3

In [72]:
tag = pd.read_csv('tag_tag.csv')
tag_dic = tag.set_index('소분류').T.to_dict()

def make_tag_1(x):
    return x['tag'][0]

def make_tag_2(x):
    if x['tag_len'] >= 2:
        return x['tag'][1]
    else:
        return x['tag'][0]

def make_tag_3(x):
    if x['tag_len'] >= 3:
        return x['tag'][2]
    else:
        return x['tag'][0]


rest['tag_1'] = rest.apply(lambda x : make_tag_1(x), axis = 1)
rest['tag_2'] = rest.apply(lambda x : make_tag_2(x), axis = 1)
rest['tag_3'] = rest.apply(lambda x : make_tag_3(x), axis = 1)

rest['tag_1'] = rest['tag_1'].apply(lambda x : tag_dic[x]['대분류'])
rest['tag_2'] = rest['tag_2'].apply(lambda x : tag_dic[x]['대분류'])
rest['tag_3'] = rest['tag_3'].apply(lambda x : tag_dic[x]['대분류'])

In [74]:
tag['대분류'].unique()

array(['한식', '일식', '디저트', '고기', '양식', '술집', '해산물', '중식', '카페', '수작업',
       '동남아음식', '음식아님', '샐러드', '분식', '음식', '패스트푸드'], dtype=object)

In [80]:
rest['tag_1'][rest['restaurant'] == '부산어묵'] = '분식'
rest['tag_2'][rest['restaurant'] == '부산어묵'] = '분식'
rest['tag_3'][rest['restaurant'] == '부산어묵'] = '분식'

rest['tag_1'][rest['restaurant'] == '하모네꼬'] = '디저트'
rest['tag_2'][rest['restaurant'] == '하모네꼬'] = '디저트'
rest['tag_3'][rest['restaurant'] == '하모네꼬'] = '디저트'

rest['tag_1'][rest['restaurant'] == '람보르끼니'] = '양식'
rest['tag_2'][rest['restaurant'] == '람보르끼니'] = '양식'
rest['tag_3'][rest['restaurant'] == '람보르끼니'] = '양식'

rest['tag_1'][rest['restaurant'] == '쟌스키친'] = '디저트'
rest['tag_2'][rest['restaurant'] == '쟌스키친'] = '디저트'
rest['tag_3'][rest['restaurant'] == '쟌스키친'] = '디저트'

In [89]:
random.seed(seed)
data['rand'] = data['rest'].apply(lambda x : random.random())
_user = data['userid'].value_counts().reset_index()
_user.columns = ['userid', 'cnt']
data = pd.merge(data, _user, how = 'left', on = 'userid')
data = data[data['cnt'] > 5].reset_index(drop = True)
data = data[~(data['userid'] == '')].reset_index(drop = True)

In [90]:
user2idx = {user:idx for idx, user in enumerate(data['userid'].unique())}
idx2user = {idx:user for idx, user in enumerate(data['userid'].unique())}

## 패딩 때문에 0 비워놓음
item2idx = {item:(idx+1) for idx, item in enumerate(data['rest'].unique())}
idx2item = {(idx+1):item for idx, item in enumerate(data['rest'].unique())}

data['user_code'] = data['userid'].map(user2idx)
data['rest_code'] = data['rest'].map(item2idx)
rest['rest_code'] = rest['url'].map(item2idx)

In [91]:
# train/ test 분할하는 부분
data = data.sort_values(['userid', 'rand']).reset_index(drop = True)
data['tem'] = 1
data['seq'] = data.groupby('userid')['tem'].apply(lambda x : x.cumsum())
data['cnt'] = data['cnt'] // 5 + 1

train = data[data['seq'] > data['cnt']]
test = data[data['seq'] <= data['cnt']]

In [92]:
print(train['user_code'].nunique())
print(test['user_code'].nunique())

382940
382940


In [95]:
train[['userid', 'rest', 'user_code', 'rest_code']].to_csv('train.csv', index = False)
test[['userid', 'rest', 'user_code', 'rest_code']].to_csv('test.csv', index = False)

user.to_csv('user.csv', index = False)
rest.to_csv('rest.csv', index = False)

In [96]:
user = train.groupby('user_code')['rest_code'].unique().to_frame().reset_index()
user['user'] = user['user_code'].map(idx2user)
user['rest_code'] = user['rest_code'].apply(lambda x : list(x))
user['rest_code'] = user['rest_code'].astype('str')
user.head()

,user_code,rest_code,user
0,0,"[14, 6, 17, 8, 12, 16, 1, 19, 20, 3, 9, 5, 10,...",5b61c7658f8242cb2a1b1028
1,1,"[25, 62, 40, 22, 28, 59, 45, 66, 46, 56, 26, 3...",5b62c91cd898db2e71efcc59
2,2,"[87, 80, 86, 88, 83, 85, 81, 77, 78]",5b62e55e10599e4793d54f8a
3,3,"[95, 96, 92, 90, 89, 93, 91]",5b62e6c46daf19a0c8d33997
4,4,"[118, 106, 133, 145, 105, 113, 119, 127, 132, ...",5b62e6e8c5d392d7522d1436
...,...,...,...
382935,382935,"[12626, 22613, 18715, 14556, 22398]",63ace40b1c31d0c76388dd4b
382936,382936,"[10425, 10433, 418, 6713, 27706, 20836, 2966]",63aec03fabbacd0006089e53
382937,382937,"[13260, 10971, 4040, 13850, 15585, 6379]",63b3caa11438fc3a9c00daab
382938,382938,"[1505, 17551, 36969, 34875]",63b877d3f70fd245ec99001b


In [105]:
rest = rest[~rest['rest_code'].isnull()]
rest['rest_code'] = rest['rest_code'].astype('int')
rest = rest[['restaurant','tag_1','url','x','y','image','rest_code']]

In [106]:
cursor.execute("CREATE TABLE rest ("
               "name VARCHAR(255),"
               "tag VARCHAR(511),"
               "url INT(127),"
               "x INT(127),"
               "y INT(127),"
               "image VARCHAR(511),"
               "rest_code INT(127))"
)
cnxn.commit()  # this commits changes to the database

a = rest.values.tolist()
for i in range(len(a)):
    a[i] = tuple(list(a[i]))

for i in tqdm(a):
    #print(i)
    cursor.execute("INSERT INTO rest VALUES (?, ?, ?, ?, ?, ?, ?)", i)
    #except: pass
cnxn.commit()

100%|██████████| 42829/42829 [00:00<00:00, 302587.19it/s]


In [149]:
# cursor.execute("DROP TABLE user")
# cnxn.commit()  # this commits changes to the database

cursor.execute("CREATE TABLE user ("
               "user_code INT(127),"
               "rest_code VARCHAR(16383),"
               "user VARCHAR(255))"
)
cnxn.commit()  # this commits changes to the database

a = user.values.tolist()
for i in range(len(a)):
    a[i] = tuple(list(a[i]))

for i in tqdm(a):
    #print(i)
    cursor.execute("INSERT INTO user VALUES (?, ?, ?)", i)
    #except: pass
cnxn.commit()

100%|██████████| 382940/382940 [00:01<00:00, 372472.02it/s]


In [150]:
select_sql = "select * from user"# where rating = 4.42"
cursor.execute(select_sql)
result = cursor.fetchall()
result

[(0,
  '[14, 6, 17, 8, 12, 16, 1, 19, 20, 3, 9, 5, 10, 11, 18]',
  '5b61c7658f8242cb2a1b1028'),
 (1,
  '[25, 62, 40, 22, 28, 59, 45, 66, 46, 56, 26, 34, 33, 21, 63, 42, 31, 51, 72, 7, 71, 44, 68, 57, 75, 70, 35, 73, 50, 60, 52, 27, 54, 55, 67, 74, 30, 65, 32, 48, 47, 49, 64, 24, 53]',
  '5b62c91cd898db2e71efcc59'),
 (2, '[87, 80, 86, 88, 83, 85, 81, 77, 78]', '5b62e55e10599e4793d54f8a'),
 (3, '[95, 96, 92, 90, 89, 93, 91]', '5b62e6c46daf19a0c8d33997'),
 (4,
  '[118, 106, 133, 145, 105, 113, 119, 127, 132, 123, 142, 134, 100, 147, 140, 104, 110, 139, 138, 143, 114, 136, 146, 103, 120, 131, 117, 121, 112, 115, 130, 108, 125, 141, 107, 124, 101, 116, 144]',
  '5b62e6e8c5d392d7522d1436'),
 (5,
  '[224, 233, 204, 229, 234, 192, 162, 168, 149, 218, 179, 191, 217, 160, 153, 239, 213, 163, 157, 220, 169, 189, 155, 210, 171, 209, 148, 203, 170, 230, 184, 176, 228, 197, 158, 166, 196, 201, 159, 182, 186, 227, 185, 206, 200, 156, 215, 175, 223, 198, 193, 202, 181, 152, 222, 172, 199, 190, 240, 22